## Entity Resolution - Deeptech Engineer Challenge


### 1. Imports

In [2]:
import pandas as pd
import re
from rapidfuzz import fuzz

### 2. Visualize dataset and preprocessing data

In [11]:
# import the dataset and show the first 5 entries
df = pd.read_parquet('./veridion_entity_resolution_challenge.snappy.parquet')
df.head()

,company_name,company_legal_names,company_commercial_names,main_country_code,main_country,main_region,main_city_district,main_city,main_postcode,main_street,...,generated_description,generated_business_tags,status,domains,all_domains,revenue,revenue_type,employee_count,employee_count_type,inbound_links_count
0,Owens Liquors,None,Owens Liquors,US,United States,South Carolina,None,Pawleys Island,29585,Ocean Highway,...,Owens Liquors is a retail establishment locate...,Retail Trade | Liquor Stores | Wine & Liquor,Active,None,None,None,None,None,None,None
1,Club Tarneit,None,Club Tarneit,AU,Australia,Victoria,Tarneit,City Of Wyndham,3029,None,...,None,None,Active,None,None,None,None,9,extracted,None
2,AAA Auto Otrokovice Zlín,None,AAA Auto Otrokovice Zlín,CZ,Czechia,Zlín,Kvítkovice U Otrokovic,Otrokovice,765 02,Zlínská,...,AAA Auto Otrokovice (Zlín) is a car dealership...,In-store Shopping | Investment Management Serv...,Active,None,None,None,None,None,None,None
3,Gisinger GmbH,Gisinger GmbH,None,DE,Germany,Baden-Württemberg,None,Ühlingen-Birkendorf,79777,Berauer Straße,...,None,None,Active,None,None,None,None,None,None,None
4,Kasana Life,None,Kasana Life,US,United States,Connecticut,None,Litchfield,06759,None,...,None,None,Active,None,None,None,None,None,None,None


In [12]:
# normalize the name of the company names
def normalize_name(name):
    if pd.isnull(name):
        return ""
    return re.sub(r'[^a-z0-9]', '', name.lower())

df['normalized_name'] = df['company_name'].apply(normalize_name)
df.head()

,company_name,company_legal_names,company_commercial_names,main_country_code,main_country,main_region,main_city_district,main_city,main_postcode,main_street,...,generated_business_tags,status,domains,all_domains,revenue,revenue_type,employee_count,employee_count_type,inbound_links_count,normalized_name
0,Owens Liquors,None,Owens Liquors,US,United States,South Carolina,None,Pawleys Island,29585,Ocean Highway,...,Retail Trade | Liquor Stores | Wine & Liquor,Active,None,None,None,None,None,None,None,owensliquors
1,Club Tarneit,None,Club Tarneit,AU,Australia,Victoria,Tarneit,City Of Wyndham,3029,None,...,None,Active,None,None,None,None,9,extracted,None,clubtarneit
2,AAA Auto Otrokovice Zlín,None,AAA Auto Otrokovice Zlín,CZ,Czechia,Zlín,Kvítkovice U Otrokovic,Otrokovice,765 02,Zlínská,...,In-store Shopping | Investment Management Serv...,Active,None,None,None,None,None,None,None,aaaautootrokovicezln
3,Gisinger GmbH,Gisinger GmbH,None,DE,Germany,Baden-Württemberg,None,Ühlingen-Birkendorf,79777,Berauer Straße,...,None,Active,None,None,None,None,None,None,None,gisingergmbh
4,Kasana Life,None,Kasana Life,US,United States,Connecticut,None,Litchfield,06759,None,...,None,Active,None,None,None,None,None,None,None,kasanalife


### 3. Adding columns and groupping unique and duplicate company names

In [13]:
# group the duplicate ones
groups = {}
group_id = 0
name_to_group = {}

unique_names = df['normalized_name'].unique()
for name in unique_names:
    assigned = False
    for rep in groups.keys():
        if fuzz.ratio(name, rep) > 90:
            name_to_group[name] = groups[rep]
            assigned = True
            break
    if not assigned:
        groups[name] = group_id
        name_to_group[name] = group_id
        group_id += 1

df['company_group'] = df['normalized_name'].map(name_to_group)
df.head()

,company_name,company_legal_names,company_commercial_names,main_country_code,main_country,main_region,main_city_district,main_city,main_postcode,main_street,...,status,domains,all_domains,revenue,revenue_type,employee_count,employee_count_type,inbound_links_count,normalized_name,company_group
0,Owens Liquors,None,Owens Liquors,US,United States,South Carolina,None,Pawleys Island,29585,Ocean Highway,...,Active,None,None,None,None,None,None,None,owensliquors,0
1,Club Tarneit,None,Club Tarneit,AU,Australia,Victoria,Tarneit,City Of Wyndham,3029,None,...,Active,None,None,None,None,9,extracted,None,clubtarneit,1
2,AAA Auto Otrokovice Zlín,None,AAA Auto Otrokovice Zlín,CZ,Czechia,Zlín,Kvítkovice U Otrokovic,Otrokovice,765 02,Zlínská,...,Active,None,None,None,None,None,None,None,aaaautootrokovicezln,2
3,Gisinger GmbH,Gisinger GmbH,None,DE,Germany,Baden-Württemberg,None,Ühlingen-Birkendorf,79777,Berauer Straße,...,Active,None,None,None,None,None,None,None,gisingergmbh,3
4,Kasana Life,None,Kasana Life,US,United States,Connecticut,None,Litchfield,06759,None,...,Active,None,None,None,None,None,None,None,kasanalife,4


In [ ]:
# show the group ones
grouped_df = df.groupby('company_group').agg({
    'company_name': lambda x: list(x)
})
grouped_df.sample(15)

,company_name
company_group,
9691,[UnipolRental S.p.A. - Verona]
6578,[DW Don's Automotive Group]
6352,"[Juanitos, Juanitos, Juanitos, Juanitos]"
12080,"[Wei Yeong, Wei Yeong]"
9448,"[First Class Block Paving Cleaning, First Clas..."
14364,[Heritage Coffee Roasting Co - Glacier Cafe]
305,"[Hoeve de Groenzoom, Hoeve de Groenzoom, Hoeve..."
3648,"[Redmond High School, Redmond High School]"
6245,"[Koch & Partner, Koch & Partner, Koch & Partne..."


### 4. Save the new dataset

In [ ]:
# result_all_columns
df.to_parquet('result_all_columns.parquet') 

In [21]:
# result_just_groupped
grouped_df.to_parquet('result_just_groupped.parquet') 